In [1]:
# Importar librerías
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [2]:
# Definir la red neuronal para clasificación
def create_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Para clasificación binaria
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [3]:
# Configurar directorios de datos
base_dir = 'pictures'  # Directorio raíz donde están las imágenes
print("Directorio verificado")


Directorio verificado


In [4]:
# Preparar generadores de imágenes
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 18 images belonging to 2 classes.
Found 4 images belonging to 2 classes.


In [5]:
# Entrenar el modelo
model = create_model()
model.fit(train_generator, validation_data=validation_generator, epochs=10)

c:\Users\rojas\Desktop\Ev4 Jose Rojas\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\rojas\Desktop\Ev4 Jose Rojas\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.5000 - loss: 0.6894 - val_accuracy: 0.5000 - val_loss: 2.6174
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step - accuracy: 0.4444 - loss: 2.4476 - val_accuracy: 0.5000 - val_loss: 3.0842
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 296ms/step - accuracy: 0.5000 - loss: 2.6127 - val_accuracy: 0.5000 - val_loss: 1.3942
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 320ms/step - accuracy: 0.3889 - loss: 1.4928 - val_accuracy: 0.5000 - val_loss: 0.6839
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step - accuracy: 0.5556 - loss: 0.9784 - val_accuracy: 0.5000 - val_loss: 0.7975
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step - accuracy: 0.4444 - loss: 0.9343 - val_accuracy: 0.5000 - val_loss: 0.7767
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step - accuracy: 0.7778 - loss: 0.5853 - val_accuracy: 0.5000 - val_loss: 0.7311
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step - accuracy: 0.3889 - loss: 0.7259 - val_accuracy: 0.5000 - val_loss: 0.

In [6]:
# Guardar el modelo
model.save('label_verification_model.h5')
print("Modelo entrenado y guardado como 'label_verification_model.h5'.")

Modelo entrenado y guardado como 'label_verification_model.h5'.


In [7]:
# Función para preprocesar imágenes
def preprocess_image(img):
    img_resized = cv2.resize(img, (128, 128))
    img_normalized = img_resized / 255.0
    return np.expand_dims(img_normalized, axis=0)

In [8]:
# Predecir y verificar etiquetas en una imagen
def predict_label(image_path):
    img = cv2.imread(image_path)
    original_img = img.copy()
    processed_img = preprocess_image(img)
    prediction = model.predict(processed_img)

    if prediction >= 0.5:
        label = "Valid Label"
        color = (0, 255, 0)  # Verde
    else:
        label = "Invalid Label"
        color = (0, 0, 255)  # Rojo

    cv2.rectangle(original_img, (10, 10), (img.shape[1] - 10, img.shape[0] - 10), color, 2)
    cv2.putText(original_img, label, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    cv2.imshow('Result', original_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [9]:
# predicción
predict_label('imagen.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
